In [14]:
import os
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB

In [15]:
# ─── CONFIG ──────────────────────────────────────────────────────────────────
cores    = 14
DATA_DIR  = f"../results/{cores}/"
G_FILE   = "G_matrix.csv"
C_FILE   = "C_vector.csv"
B_FILE   = "b_vector.csv"
P_FILE   = "Pij.csv"
F_FILE   = "fij.csv"
T_FILE   = "Tprev_vec.csv"

N_JOBS = 10     # total jobs to schedule
T_MAX  = 85.0   # max allowable temperature

# ─── LOAD THERMAL MODEL PARAMETERS ──────────────────────────────────────────
# G matrix (N×N)
g_path = os.path.join(DATA_DIR, G_FILE)
G_df = pd.read_csv(g_path, header=0)
G_mat = G_df.values

# C vector (length N)
c_path = os.path.join(DATA_DIR, C_FILE)
# read as single-column CSV without header
C_vec = pd.read_csv(c_path, header=None).iloc[1:,0].values

# b vector (length N)
b_path = os.path.join(DATA_DIR, B_FILE)
b_vec = pd.read_csv(b_path, header=None).iloc[1:,0].values

# ─── LOAD SCHEDULING DATA ───────────────────────────────────────────────────
# Power map Pij (N×M)
P_mat = pd.read_csv(os.path.join(DATA_DIR, P_FILE), index_col=0).values
# Performance map fij (N×M)
f_mat = pd.read_csv(os.path.join(DATA_DIR, F_FILE), index_col=0).values
# Previous temperatures Tprev (length N)
T_prev = pd.read_csv(os.path.join(DATA_DIR, T_FILE), header=None).iloc[:,0].values

# Idle power used in proxy (must match training)
BAR_P = 2.51  # watts at idle

# Dimensions inferred
P_rows, M = P_mat.shape
N = G_mat.shape[0]

if P_rows != N:
    raise ValueError('Dimension mismatch: G_mat cores vs Pij rows')
# Validate lengths of C_vec, b_vec, T_prev
if C_vec.shape[0] != N or b_vec.shape[0] != N or T_prev.shape[0] != N:
    raise ValueError('Mismatch in vector lengths vs N cores')
    
# Compute ambient term: T_a = C * T_prev + b
T_a = C_vec * T_prev + b_vec

In [16]:
# ─── BUILD GUROBI MODEL ───────────────────────────────────────────────────────
model = gp.Model("Thermal_Scheduler")

# Decision variables
x = model.addVars(N, M, vtype=GRB.BINARY, name="x")  # x[i,j] = 1 if core i at freq j
y = model.addVars(N, vtype=GRB.BINARY, name="y")    # y[i] = 1 if core i is idle

# Objective: maximize total throughput
model.setObjective(gp.quicksum(f_mat[i, j] * x[i, j] for i in range(N) for j in range(M)), GRB.MAXIMIZE)

# ─── CONSTRAINTS ─────────────────────────────────────────────────────────────
# 1) Exact number of jobs scheduled
model.addConstr(x.sum('*','*') == N_JOBS, name="job_count")

# 2) Idle cores count
model.addConstr(y.sum() == N - N_JOBS, name="idle_count")

# 3) Each core either idle or one frequency
for i in range(N):
    model.addConstr(gp.quicksum(x[i, j] for j in range(M)) + y[i] == 1, name=f"assign_core_{i}")
    
# 4) Thermal safety per core ℓ
for ell in range(N):
    expr = gp.LinExpr()
    for i in range(N):
        # steady-state power on core i: BAR_P*y[i] + sum_j P_ij*x[i,j]
        pow_i = BAR_P * y[i] + gp.quicksum(P_mat[i, j] * x[i, j] for j in range(M))
        expr += G_mat[ell, i] * pow_i
    expr += T_a[ell]
    model.addConstr(expr <= T_MAX, name=f"thermal_core_{ell}")

# ─── SOLVE ───────────────────────────────────────────────────────────────────
model.Params.LogToConsole = 1
model.optimize()

# ─── EXTRACT AND DISPLAY SOLUTION ─────────────────────────────────────────────
if model.status == GRB.OPTIMAL:
    total_throughput = model.objVal
    print(f"Optimal Total Throughput: {total_throughput:.2f}")
    active_cores = []
    idle_cores = []
    for i in range(N):
        assigned = [j for j in range(M) if x[i,j].X > 0.5]
        if assigned:
            active_cores.append((i, assigned[0]))
        else:
            idle_cores.append(i)
    print("\nCore Assignments:")
    for i, j in active_cores:
        print(f"  Core {i:02d}: Frequency Level {j}")
    print("\nIdle Cores:")
    print("  ", idle_cores)
else:
    print(f"No optimal solution found (status {model.status})")

Set parameter LogToConsole to value 1
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.4.0 24E263)

CPU model: Apple M4 Pro
Thread count: 14 physical cores, 14 logical processors, using up to 14 threads

Optimize a model with 226 rows, 448 columns and 51072 nonzeros
Model fingerprint: 0x3942a464
Variable types: 0 continuous, 448 integer (448 binary)
Coefficient statistics:
  Matrix range     [6e-08, 3e+02]
  Objective range  [3e+07, 3e+09]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-03, 2e+02]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective 6.352987e+09
Presolve removed 2 rows and 6 columns
Presolve time: 0.07s
Presolved: 224 rows, 442 columns, 40551 nonzeros
Found heuristic solution: objective 2.400156e+10
Variable types: 0 continuous, 442 integer (442 binary)

Root relaxation: objective 2.544835e+10, 76 iterations, 0.00 seconds (0.00 work units)

    No

In [17]:
model.write("model.lp")